In [15]:
from pydantic_ai import Agent
import nest_asyncio
from pathlib import Path
from dotenv import load_dotenv
import chromadb
import pandas as pd
import re
from src.utilities.constants import NUMBER_SIMILARITY_REULTS, USER_QUERY_COLLECTION_NAME, SPARQL_QUERY_COLLECTION_NAME
nest_asyncio.apply()
load_dotenv()

True

In [22]:
def update_collection(create_collections : bool):
    file_path = Path.cwd() / "querySparql.xlsx"
    df = pd.read_excel(file_path, sheet_name="Foglio1", names=["question", "sparql_query"])
    ids = []
    queries = []
    for row in df.itertuples():
        user_query = re.sub("\n", " ", row[1])
        user_query = re.sub("\t", " ", user_query)

        s_query = re.sub("\n", " ", row[2])
        s_query = re.sub("\t", " ", s_query)

        queries.append({"user_query": user_query, "sparql_query": s_query})
        ids.append(f"ID_{row[0]}")

    
    client = chromadb.HttpClient(host='localhost', port=8000)
    if create_collections:
        client.create_collection(USER_QUERY_COLLECTION_NAME)
        client.create_collection(SPARQL_QUERY_COLLECTION_NAME)

    u_query_collection = client.get_collection(name=USER_QUERY_COLLECTION_NAME)
    s_query_collection = client.get_collection(name=SPARQL_QUERY_COLLECTION_NAME)


    for i, query in zip(ids, queries):
        u_query_collection.add(
            documents=query.get("user_query"),
            ids=i
        )
        s_query_collection.add(
            documents=query.get("sparql_query"),
            ids=i
        )

    return f"Collections updated\nCollections size: {u_query_collection.count(), s_query_collection.count()}"

update_collection(create_collections=False)

'Collections updated\nCollections size: (88, 88)'